In [53]:
import pandas as pd


bridges_path = "df_bridges_data.csv"


# Load the Excel file into a DataFrame
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [54]:
import geopandas as gpd
# Replace the file path with your actual file path
region_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
# region_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
region_gdf = gpd.read_file(region_path)


districts_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # doma
# districts_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
districts_gdf = gpd.read_file(districts_path)


# Replace the file path with your actual file path
file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma
# file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # robota


# Load the GeoJSON file into a GeoDataFrame
settle_gdf = gpd.read_file(file_path)

# Define the types to exclude
types_to_exclude = ['country', 'airport', 'Psie skaly',
                    'county', 'Drevenný kostol', 'Vodopád']

# Create a boolean mask to select rows where the 'type' column is not in types_to_exclude
# and the 'population' column is not null
mask = (~settle_gdf['type'].isin(types_to_exclude)) & (
    settle_gdf['population'] > 0)


# Apply the mask to filter the GeoDataFrame in place
settle_gdf = settle_gdf[mask]

# Drop rows where the 'name' column is "Bratislava" or "Košice" from settle_gdf
settle_gdf = settle_gdf[~settle_gdf["name"].isin(["Bratislava", "Košice"])]

settle_gdf["osm_id"] = settle_gdf["osm_id"].astype(int)

In [55]:
# Load the CSV file into a DataFrame
df_distances = pd.read_csv('df_bridges_settlements_distances.csv')

In [56]:
df_distances.head(7)

,osm_id,name,M5897,M1979,M6525,M4584,M3053,M1833,M1055,M1839,...,M10157,M10121,M10150,M10149,M10080,M9963,M9965,M9966,M9967,M9969
0,15243569,Hronský Beňadik,117.0,118.5,119.0,119.1,119.7,120.1,119.1,119.1,...,210.5,122.3,163.4,179.1,298.0,73.4,74.3,75.1,76.2,76.6
1,26035645,Beluša,93.8,98.2,104.5,106.2,108.7,109.3,111.0,111.9,...,198.4,133.4,162.0,178.8,302.8,110.1,111.5,112.4,115.2,116.4
2,26035646,Pusté Pole,280.3,284.3,289.4,290.7,292.8,293.4,294.2,294.9,...,13.8,77.7,35.2,25.0,117.0,129.9,129.8,129.3,129.9,130.3
3,26035647,Hažín nad Cirochou,355.3,358.8,363.1,364.2,365.9,366.4,366.8,367.3,...,79.9,138.3,104.0,87.1,39.1,185.8,185.1,184.3,183.7,183.5
4,26035648,Demandice,143.1,144.2,143.6,143.4,143.6,143.9,142.4,142.3,...,211.2,120.0,162.5,176.8,290.2,67.8,68.1,68.7,68.6,68.5
5,26035649,Laskár,130.0,133.8,138.8,140.2,142.2,142.8,143.8,144.5,...,161.9,91.9,122.3,139.2,263.5,72.1,73.6,74.4,77.4,78.7
6,26035650,Vyškovce nad Ipľom,152.8,153.8,153.0,152.7,152.9,153.1,151.5,151.3,...,211.3,119.9,162.4,176.2,287.2,67.7,67.8,68.3,67.7,67.4


In [57]:
# Create a new DataFrame by merging df_distances_within_50km with settle_gdf on the 'osm_id' column
df_distances = pd.merge(df_distances, settle_gdf[[
                     'osm_id', 'population']], on='osm_id', how='left')

In [58]:
df_distances.columns

Index(['osm_id', 'name', 'M5897', 'M1979', 'M6525', 'M4584', 'M3053', 'M1833',
       'M1055', 'M1839',
       ...
       'M10121', 'M10150', 'M10149', 'M10080', 'M9963', 'M9965', 'M9966',
       'M9967', 'M9969', 'population'],
      dtype='object', length=8128)

In [59]:
df_bridges_all["ID_mosta"]

0       M5897
1       M1979
2       M6525
3       M4584
4       M3053
        ...  
8120    M9963
8121    M9965
8122    M9966
8123    M9967
8124    M9969
Name: ID_mosta, Length: 8125, dtype: object

In [60]:
d1 = 4
d2 = 12
d3 = 28

# Initialize new columns in df_bridges_all
df_bridges_all['population_1'] = 0
df_bridges_all['population_2'] = 0
df_bridges_all['population_3'] = 0

# Iterate over each row in df_bridges_all to compute population sums
for index, bridge in df_bridges_all.iterrows():
    bridge_id = bridge['ID_mosta']
    # Extract distances to this bridge from all settlements
    distances_to_bridge = df_distances[bridge_id]

    # Calculate populations within specified distance ranges
    population_1 = df_distances.loc[distances_to_bridge <=
                                    d1, 'population'].sum()
    population_2 = df_distances.loc[(distances_to_bridge > d1) & (
        distances_to_bridge <= d2), 'population'].sum()
    population_3 = df_distances.loc[(distances_to_bridge > d2) & (
        distances_to_bridge <= d3), 'population'].sum()

    # Update df_bridges_all with calculated populations
    df_bridges_all.at[index, 'population_1'] = population_1
    df_bridges_all.at[index, 'population_2'] = population_2
    df_bridges_all.at[index, 'population_3'] = population_3

# After the loop, df_bridges_all will have the new columns populated with the respective population sums

In [61]:
df_bridges_all[["ID_mosta", 'Zemepisná_šírka',
                'Zemepisná_dĺžka', 'population_1', 'population_2', 'population_3']]

,ID_mosta,Zemepisná_šírka,Zemepisná_dĺžka,population_1,population_2,population_3
0,M5897,48.790152,17.123319,11627,14247,55821
1,M1979,48.757287,17.079098,0,28109,55610
2,M6525,48.684515,17.031675,4121,20034,88258
3,M4584,48.663688,17.020574,4121,18504,89567
4,M3053,48.639120,17.001660,4121,16041,92434
...,...,...,...,...,...,...
8120,M9963,48.474460,19.533000,4176,36450,83323
8121,M9965,48.465050,19.547370,5037,14359,105217
8122,M9966,48.462190,19.559300,6827,14254,102607
8123,M9967,48.438690,19.579420,6776,17295,113848


In [62]:
df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'popula